In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import glob

client_credentials_manager = SpotifyClientCredentials(
        "26399552a8ce4d1285397254189cac50",
        "fdacfffa2dd34dbeb127dedb459f7ea3")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [3]:
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Oceania + SouthAmerica
print(len(countriesOfInterest))
for year in range(2019,2000, -1):
    tmstamp = str(year) + "-06-15T12:00:00"
    for countryCode in countriesOfInterest:
        if len(glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")) == 0:
            country = sp.featured_playlists(country = countryCode, limit = 50)
            plist = country["playlists"]["items"]

            plist = pd.DataFrame.from_dict(plist)
        
            print(countryCode, "has", len(plist), "playlists in ", str(year))
            plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv", index = False)
            for uri in plist["uri"]:
                currplist = sp.playlist(uri)
                trackURI = []
                for track in currplist["tracks"]["items"]:
                    try:
                        trackURI = trackURI + [track["track"]["uri"]]
                    except:
                        continue
                print("Getting track features for", uri, "in", countryCode)
                try:
                    trackData = getTrackFeatures(trackURI)
                    trackData.to_csv("Raw Track Data\\" + countryCode + "_" + str(year) + "_" + uri[17:] + ".csv", index = False)
                except:
                    continue
            

36
TN has 15 playlists in  2016
Getting track features for spotify:playlist:37i9dQZF1DX4NdDHVFmygr in TN
Getting track features for spotify:playlist:37i9dQZF1DX7ZqKP81pHlu in TN
Getting track features for spotify:playlist:37i9dQZF1DX9HeUXWsJonl in TN
Getting track features for spotify:playlist:37i9dQZF1DX6qjeIoR93rs in TN
Getting track features for spotify:playlist:37i9dQZF1DWXJnW46G73SM in TN
Getting track features for spotify:playlist:37i9dQZF1DWVSIz2AGspV4 in TN
Getting track features for spotify:playlist:37i9dQZF1DWUyqO4fBh3ob in TN
Getting track features for spotify:playlist:37i9dQZF1DXaywRlmHaP59 in TN
Getting track features for spotify:playlist:37i9dQZF1DX15iTqq2X8NE in TN
Getting track features for spotify:playlist:37i9dQZF1DWWQRKXabpuS2 in TN
Getting track features for spotify:playlist:37i9dQZF1DX2LJXoyg2yd7 in TN
Getting track features for spotify:playlist:37i9dQZF1DWUlTJaFU1ddx in TN
Getting track features for spotify:playlist:37i9dQZF1DXcZXC7Soywtq in TN
Getting track featu

Getting track features for spotify:playlist:37i9dQZF1DX0s5kDXi1oC5 in JO
Getting track features for spotify:playlist:37i9dQZF1DXe3aCmUoBd8n in JO
Getting track features for spotify:playlist:37i9dQZF1DX0ysO3y1BFXN in JO
KW has 20 playlists in  2016
Getting track features for spotify:playlist:37i9dQZF1DWWgW3m1g7KmT in KW
Getting track features for spotify:playlist:37i9dQZF1DWXfv2VdqxYl4 in KW
Getting track features for spotify:playlist:37i9dQZF1DWYHBENPjyDpc in KW
Getting track features for spotify:playlist:37i9dQZF1DWU486KSiznWZ in KW
Getting track features for spotify:playlist:37i9dQZF1DWTvNyxOwkztu in KW
Getting track features for spotify:playlist:37i9dQZF1DWZKucvJjqs4a in KW
Getting track features for spotify:playlist:37i9dQZF1DX0ysO3y1BFXN in KW
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in KW
Getting track features for spotify:playlist:37i9dQZF1DX1fzTwl8xjJf in KW
Getting track features for spotify:playlist:37i9dQZF1DXdj3JotSexPH in KW
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DWWhuJcTXO5hW in QA
Getting track features for spotify:playlist:37i9dQZF1DX8DbV65q7h3u in QA
Getting track features for spotify:playlist:37i9dQZF1DWTTCCEjVC99N in QA
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in QA
Getting track features for spotify:playlist:37i9dQZF1DX5lckay3OqOG in QA
Getting track features for spotify:playlist:37i9dQZF1DXa7JWqw8BIJc in QA
SA has 18 playlists in  2016
Getting track features for spotify:playlist:37i9dQZF1DWT5MrZnPU1zD in SA
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in SA
Getting track features for spotify:playlist:37i9dQZF1DWWgW3m1g7KmT in SA
Getting track features for spotify:playlist:37i9dQZF1DWU486KSiznWZ in SA
Getting track features for spotify:playlist:37i9dQZF1DWXfv2VdqxYl4 in SA
Getting track features for spotify:playlist:37i9dQZF1DXaFm6IhSHlyi in SA
Getting track features for spotify:playlist:37i9dQZF1DXa7FSukIBTlS in SA
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DX5D8j6p4CXZT in NZ
Getting track features for spotify:playlist:37i9dQZF1DXc78A1OIheFs in NZ
Getting track features for spotify:playlist:37i9dQZF1DXdxjdZwLamNx in NZ
Getting track features for spotify:playlist:37i9dQZF1DX4sWSpwq3LiO in NZ
Getting track features for spotify:playlist:37i9dQZF1DX3Liv35LmYo1 in NZ
Getting track features for spotify:playlist:37i9dQZF1DXamBAn43YNB9 in NZ
Getting track features for spotify:playlist:37i9dQZF1DWZd79rJ6a7lp in NZ
Getting track features for spotify:playlist:37i9dQZF1DX5eHUc6ZpsPh in NZ
Getting track features for spotify:playlist:37i9dQZF1DX7XctuunSzQQ in NZ
AR has 17 playlists in  2016
Getting track features for spotify:playlist:37i9dQZF1DXavAgzH4TQLa in AR
Getting track features for spotify:playlist:37i9dQZF1DWVskFRGurTfg in AR
Getting track features for spotify:playlist:37i9dQZF1DWUlCmB8llCTB in AR
Getting track features for spotify:playlist:37i9dQZF1DWUmxBdWX3Pp4 in AR
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DX7aAuYd7Jogj in UY
Getting track features for spotify:playlist:37i9dQZF1DWZeuoGxfO4e2 in UY
Getting track features for spotify:playlist:37i9dQZF1DXco4ODkIraMD in UY
Getting track features for spotify:playlist:37i9dQZF1DWY5LGZYBBHHz in UY
Getting track features for spotify:playlist:37i9dQZF1DX20S3TU4dlyR in UY
Getting track features for spotify:playlist:37i9dQZF1DX9o4SIlW5lWs in UY
DZ has 15 playlists in  2015
Getting track features for spotify:playlist:37i9dQZF1DX403FtPdcGpX in DZ
Getting track features for spotify:playlist:37i9dQZF1DX4lqD8KeW3DZ in DZ
Getting track features for spotify:playlist:37i9dQZF1DXb7WmotStdsj in DZ
Getting track features for spotify:playlist:37i9dQZF1DWWhuJcTXO5hW in DZ
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in DZ
Getting track features for spotify:playlist:37i9dQZF1DX5lckay3OqOG in DZ
Getting track features for spotify:playlist:37i9dQZF1DX9ND1QF5hZNF in DZ
Getting track features

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\jgomezm\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-94c4c5066dae>", line 27, in <module>
    currplist = sp.playlist(uri)
  File "C:\Users\jgomezm\AppData\Local\Continuum\anaconda3\lib\site-packages\spotipy\client.py", line 430, in playlist
    return self._get("playlists/%s" % (plid), fields=fields, market=market)
  File "C:\Users\jgomezm\AppData\Local\Continuum\anaconda3\lib\site-packages\spotipy\client.py", line 193, in _get
    return self._internal_call("GET", url, payload, kwargs)
  File "C:\Users\jgomezm\AppData\Local\Continuum\anaconda3\lib\site-packages\spotipy\client.py", line 150, in _internal_call
    method, url, headers=headers, proxies=self.proxies, **args
  File "C:\Users\jgomezm\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\sessions.py", line 533, in request
  

KeyboardInterrupt: 

In [ ]:
glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")

In [ ]:
for year in range(2019,2000, -1):
    print(year)

In [ ]:
glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")